In [1]:
from MovieRecommend import Movies

In [2]:
mov = Movies()

Loading saved movie database...


In [3]:
mov.print_files()

s Stoller (sc...  2005         6.2   
85   Mario Puzo (screenplay by), Francis Ford Coppo...  1972         9.2   
86   Francis Ford Coppola (screenplay by), Mario Pu...  1974           9   
87   Terry Rossio (screenplay), Ted Elliott (screen...  2000         6.9   
88   Jack Bernstein (story), Jack Bernstein (screen...  1994         6.9   
89         Jack Bernstein (characters), Steve Oedekerk  1995         6.4   
9                                         George Gallo  1988         7.5   
90   John Wagner ('Judge Dredd' created by), Carlos...  2012         7.1   
91   Andrew Stanton (original story by), Andrew Sta...  2016         7.3   
92               John Hughes (characters), John Hughes  1992         6.8   
93                                     Russell Gewirtz  2006         7.6   
94   Callie Kloves (screenplay by), Rudyard Kipling...  2018         6.5   
95   Todd Komarnicki, Chesley Sullenberger (based o...  2016         7.4   
96   Charlie Kaufman (story), Michel Gondry (story

In [4]:
mov.preview_files()

                                                Actors  \
0    Michelle Williams, Christopher Plummer, Mark W...   
1    Dwayne Johnson, Kevin Hart, Jack Black, Karen ...   
10   Chadwick Boseman, Michael B. Jordan, Lupita Ny...   
100  Margot Robbie, Sebastian Stan, Allison Janney,...   
101  Jennifer Lawrence, Chris Pratt, Michael Sheen,...   

                                                Awards     BoxOffice  \
0       Nominated for 1 Oscar. Another 14 nominations.   $25,113,707   
1                             6 wins & 19 nominations.  $404,540,171   
10   Won 3 Oscars. Another 109 wins & 274 nominations.  $700,426,566   
100    Won 1 Oscar. Another 44 wins & 125 nominations.   $30,014,539   
101  Nominated for 2 Oscars. Another 1 win & 10 nom...  $100,014,699   

            Director                                       Genre Metascore  \
0       Ridley Scott  Biography, Crime, Drama, Mystery, Thriller        72   
1        Jake Kasdan          Action, Adventure, Comedy, Fanta

In [5]:
new_df = mov.get_df()

In [6]:
mov.create_features()

In create features
Writer column cleaned and assigned
Dataframe assigned for features
Plot lemma column created
Form columns split
Created new dataframes of split columns
Joining dataframes
Dataframes joined


In [7]:
mov.preview_files()

                                                Actors  \
0    [michellewilliams, christopherplummer, markwah...   
1    [dwaynejohnson, kevinhart, jackblack, karengil...   
10   [chadwickboseman, michaelb.jordan, lupitanyong...   
100  [margotrobbie, sebastianstan, allisonjanney, j...   
101  [jenniferlawrence, chrispratt, michaelsheen, l...   

                                                Awards     BoxOffice  \
0       Nominated for 1 Oscar. Another 14 nominations.   $25,113,707   
1                             6 wins & 19 nominations.  $404,540,171   
10   Won 3 Oscars. Another 109 wins & 274 nominations.  $700,426,566   
100    Won 1 Oscar. Another 44 wins & 125 nominations.   $30,014,539   
101  Nominated for 2 Oscars. Another 1 win & 10 nom...  $100,014,699   

             Director                                         Genre Metascore  \
0       [ridleyscott]  [biography, crime, drama, mystery, thriller]        72   
1        [jakekasdan]          [action, adventure, comed

In [8]:
df = mov.get_df()

In [9]:
mov.print_files()

    other       other     other   
84         other             other              other       other     other   
85         other             other              other       other     other   
86         other             other              other       other     other   
87         other             other              other       other     other   
88         other             other              other       other     other   
89         other             other              other       other     other   
9          other             other              other       other     other   
90         other             other              other       other     other   
91         other             other              other       other     other   
92         other             other              other       other     other   
93         other             other              other       other     other   
94         other             other              other       other     other   
95         other 

In [10]:
print(df.columns)

Index(['Actors', 'Awards', 'BoxOffice', 'Director', 'Genre', 'Metascore',
       'Plot', 'Poster', 'Production', 'Rated', 'Ratings', 'Released',
       'Runtime', 'Title', 'Writer', 'Year', 'imdbRating', 'Format',
       'Days to View', 'Days Since Release', 'True Rating', 'Plot Lemma',
       'Genre_0', 'Genre_1', 'Genre_2', 'Genre_3', 'Genre_4', 'Genre_5',
       'Genre_6', 'Genre_7', 'Actors_0', 'Actors_1', 'Actors_2', 'Actors_3',
       'Director_0', 'Director_1', 'Director_2', 'Director_3', 'Production_0',
       'Production_1', 'Production_2', 'Production_3', 'Production_4',
       'Production_5', 'Production_6', 'Production_7', 'Production_8',
       'Production_9', 'Production_10', 'Writer_0', 'Writer_1', 'Writer_2',
       'Writer_3', 'Writer_4', 'Writer_5', 'Writer_6', 'Writer_7', 'Writer_8',
       'Writer_9', 'Writer_10', 'Writer_11', 'Writer_12', 'Writer_13',
       'Writer_14', 'Writer_15'],
      dtype='object')
